In [16]:
from pulp import *
import pandas as pd

In [17]:
human = pd.read_csv("datasets/simple_hr/hr_data - human.csv")
section = pd.read_csv("datasets/simple_hr/hr_data - section.csv")

In [18]:
human.describe()

id       cost    skill_a    skill_b   skill_c    skill_d
count  20.00000  20.000000  20.000000  20.000000  20.00000  20.000000
mean   10.50000   3.650000   0.400000   0.300000   0.35000   0.450000
std     5.91608   1.631112   0.502625   0.470162   0.48936   0.510418
min     1.00000   1.000000   0.000000   0.000000   0.00000   0.000000
25%     5.75000   2.000000   0.000000   0.000000   0.00000   0.000000
50%    10.50000   3.500000   0.000000   0.000000   0.00000   0.000000
75%    15.25000   5.000000   1.000000   1.000000   1.00000   1.000000
max    20.00000   7.000000   1.000000   1.000000   1.00000   1.000000

In [19]:
section.head()

id  num
0   1    5
1   2    5
2   3    4
3   4    3
4   5    3

In [20]:
m = LpProblem() # 部署ごとのコストを最小化する
# 最小化
for i in range(0, len(section)):
    human["section_%d"%i] = [LpVariable("v%d_%d"%(i,j), cat="Integer", upBound=1, lowBound=0) for j in human.index]
human.head()

id  cost  skill_a  skill_b  skill_c  skill_d section_0 section_1 section_2  \
0   1     5        1        1        0        0      v0_0      v1_0      v2_0   
1   2     3        0        0        1        0      v0_1      v1_1      v2_1   
2   3     4        0        1        0        1      v0_2      v1_2      v2_2   
3   4     2        1        0        0        0      v0_3      v1_3      v2_3   
4   5     5        0        0        1        1      v0_4      v1_4      v2_4   

  section_3 section_4  
0      v3_0      v4_0  
1      v3_1      v4_1  
2      v3_2      v4_2  
3      v3_3      v4_3  
4      v3_4      v4_4

In [21]:
# 最小化
cost = None
for i in range(0, len(section)):
    cost += lpDot(human.cost, human["section_%d"%i])
m+= cost

In [22]:
for index, row in section.iterrows():
    m += lpSum(human["section_%d"%index])==row.num

In [23]:
for index, row in human.iterrows():
    m += (row.section_0+row.section_1+row.section_2+row.section_3+row.section_4)==1

In [24]:
for i in range(0, len(section)):
    m += lpDot(human.skill_a, human["section_%d"%i]) >=1
    m += lpDot(human.skill_b, human["section_%d"%i]) >=1
    m += lpDot(human.skill_c, human["section_%d"%i]) >=1
    m += lpDot(human.skill_d, human["section_%d"%i]) >=1

In [25]:
pulp.LpStatus[m.solve()]

'Optimal'

In [26]:
print(m)

NoName:
MINIMIZE
5*v0_0 + 3*v0_1 + 4*v0_10 + 3*v0_11 + 2*v0_12 + 4*v0_13 + 7*v0_14 + 3*v0_15 + 2*v0_16 + 5*v0_17 + 6*v0_18 + 4*v0_19 + 4*v0_2 + 2*v0_3 + 5*v0_4 + 1*v0_5 + 2*v0_6 + 3*v0_7 + 6*v0_8 + 2*v0_9 + 5*v1_0 + 3*v1_1 + 4*v1_10 + 3*v1_11 + 2*v1_12 + 4*v1_13 + 7*v1_14 + 3*v1_15 + 2*v1_16 + 5*v1_17 + 6*v1_18 + 4*v1_19 + 4*v1_2 + 2*v1_3 + 5*v1_4 + 1*v1_5 + 2*v1_6 + 3*v1_7 + 6*v1_8 + 2*v1_9 + 5*v2_0 + 3*v2_1 + 4*v2_10 + 3*v2_11 + 2*v2_12 + 4*v2_13 + 7*v2_14 + 3*v2_15 + 2*v2_16 + 5*v2_17 + 6*v2_18 + 4*v2_19 + 4*v2_2 + 2*v2_3 + 5*v2_4 + 1*v2_5 + 2*v2_6 + 3*v2_7 + 6*v2_8 + 2*v2_9 + 5*v3_0 + 3*v3_1 + 4*v3_10 + 3*v3_11 + 2*v3_12 + 4*v3_13 + 7*v3_14 + 3*v3_15 + 2*v3_16 + 5*v3_17 + 6*v3_18 + 4*v3_19 + 4*v3_2 + 2*v3_3 + 5*v3_4 + 1*v3_5 + 2*v3_6 + 3*v3_7 + 6*v3_8 + 2*v3_9 + 5*v4_0 + 3*v4_1 + 4*v4_10 + 3*v4_11 + 2*v4_12 + 4*v4_13 + 7*v4_14 + 3*v4_15 + 2*v4_16 + 5*v4_17 + 6*v4_18 + 4*v4_19 + 4*v4_2 + 2*v4_3 + 5*v4_4 + 1*v4_5 + 2*v4_6 + 3*v4_7 + 6*v4_8 + 2*v4_9 + 0
SUBJECT TO
_C1: v0_0 + v0_1 + v

In [27]:
human['section_0_v'] = human.section_0.apply(value)
human['section_1_v'] = human.section_1.apply(value)
human['section_2_v'] = human.section_2.apply(value)
human['section_3_v'] = human.section_3.apply(value)
human['section_4_v'] = human.section_4.apply(value)

In [29]:
human[human.section_0_v==1]

id  cost  skill_a  skill_b  skill_c  skill_d section_0 section_1  \
1    2     3        0        0        1        0      v0_1      v1_1   
3    4     2        1        0        0        0      v0_3      v1_3   
6    7     2        0        0        1        0      v0_6      v1_6   
12  13     2        0        0        0        1     v0_12     v1_12   
14  15     7        1        1        0        1     v0_14     v1_14   

   section_2 section_3 section_4  section_0_v  section_1_v  section_2_v  \
1       v2_1      v3_1      v4_1          1.0          0.0          0.0   
3       v2_3      v3_3      v4_3          1.0          0.0          0.0   
6       v2_6      v3_6      v4_6          1.0          0.0          0.0   
12     v2_12     v3_12     v4_12          1.0          0.0          0.0   
14     v2_14     v3_14     v4_14          1.0          0.0          0.0   

    section_3_v  section_4_v  
1           0.0          0.0  
3           0.0          0.0  
6           0.0          0.0  
12          0.0          0.0  
14          0.0          0.0

In [30]:
human[human.section_1_v==1]

id  cost  skill_a  skill_b  skill_c  skill_d section_0 section_1  \
2    3     4        0        1        0        1      v0_2      v1_2   
9   10     2        1        0        0        0      v0_9      v1_9   
11  12     3        0        0        0        1     v0_11     v1_11   
15  16     3        1        0        0        0     v0_15     v1_15   
17  18     5        1        0        1        0     v0_17     v1_17   

   section_2 section_3 section_4  section_0_v  section_1_v  section_2_v  \
2       v2_2      v3_2      v4_2          0.0          1.0          0.0   
9       v2_9      v3_9      v4_9          0.0          1.0          0.0   
11     v2_11     v3_11     v4_11          0.0          1.0          0.0   
15     v2_15     v3_15     v4_15          0.0          1.0          0.0   
17     v2_17     v3_17     v4_17          0.0          1.0          0.0   

    section_3_v  section_4_v  
2           0.0          0.0  
9           0.0          0.0  
11          0.0          0.0  
15          0.0          0.0  
17          0.0          0.0

In [31]:
human[human.section_2_v==1]

id  cost  skill_a  skill_b  skill_c  skill_d section_0 section_1  \
0    1     5        1        1        0        0      v0_0      v1_0   
4    5     5        0        0        1        1      v0_4      v1_4   
10  11     4        0        1        0        0     v0_10     v1_10   
16  17     2        0        0        1        0     v0_16     v1_16   

   section_2 section_3 section_4  section_0_v  section_1_v  section_2_v  \
0       v2_0      v3_0      v4_0          0.0          0.0          1.0   
4       v2_4      v3_4      v4_4          0.0          0.0          1.0   
10     v2_10     v3_10     v4_10          0.0          0.0          1.0   
16     v2_16     v3_16     v4_16          0.0          0.0          1.0   

    section_3_v  section_4_v  
0           0.0          0.0  
4           0.0          0.0  
10          0.0          0.0  
16          0.0          0.0

In [32]:
human[human.section_3_v==1]

id  cost  skill_a  skill_b  skill_c  skill_d section_0 section_1  \
5    6     1        1        0        0        0      v0_5      v1_5   
13  14     4        0        0        1        0     v0_13     v1_13   
18  19     6        0        1        0        1     v0_18     v1_18   

   section_2 section_3 section_4  section_0_v  section_1_v  section_2_v  \
5       v2_5      v3_5      v4_5          0.0          0.0          0.0   
13     v2_13     v3_13     v4_13          0.0          0.0          0.0   
18     v2_18     v3_18     v4_18          0.0          0.0          0.0   

    section_3_v  section_4_v  
5           1.0          0.0  
13          1.0          0.0  
18          1.0          0.0

In [33]:
human[human.section_4_v==1]

id  cost  skill_a  skill_b  skill_c  skill_d section_0 section_1  \
7    8     3        0        0        0        1      v0_7      v1_7   
8    9     6        0        1        1        1      v0_8      v1_8   
19  20     4        1        0        0        1     v0_19     v1_19   

   section_2 section_3 section_4  section_0_v  section_1_v  section_2_v  \
7       v2_7      v3_7      v4_7          0.0          0.0          0.0   
8       v2_8      v3_8      v4_8          0.0          0.0          0.0   
19     v2_19     v3_19     v4_19          0.0          0.0          0.0   

    section_3_v  section_4_v  
7           0.0          1.0  
8           0.0          1.0  
19          0.0          1.0